# Six-bar mechanism balancing

###### Luis Eduardo Robles Jiménez

With this work, I pretend to find out the behavior of the function with different tools, in order to understand what are the most important variables, find patterns, identify overlapping zones of minima between the functions we're optimizing, graphing data, etc.

In [3]:
from BetaShF import *
from BetaShM import *

In [5]:
print(ShF(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))
print(ShM(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

1.0
None


## Mathematical analysis with sympy

### Minima

### Maxima

### Curve approximation

## Sample generation

## Dimensionality reduction

### Feature extraction

### Feature selection

## Data visualization